In [1]:
import pandas as pd
from sqlalchemy import create_engine, event
import psycopg2

### Extract CSVs into DataFrames

In [2]:
listings_file = "../Project-ETL/Resources/new_listings.csv"
listings_df = pd.read_csv(listings_file)
listings_df.head()

,id,name,description,neighbourhood_group_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,amenities,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,2318,Casa Madrona - Urban Oasis 1 block from the park!,"Gorgeous, architect remodeled, 1917 Dutch Colo...",Central Area,98122.0,House,Entire home/apt,9,2.5,4,"{Internet,Wifi,Kitchen,""Free parking on premis...",$296.00,27,100.0,strict_14_with_grace_period,0.20
1,5682,"Cozy Studio, min. to downtown -WiFi",The Cozy Studio is a perfect launchpad for you...,Delridge,98106.0,Guest suite,Entire home/apt,2,1.0,0,"{TV,Internet,Wifi,""Free street parking"",Heatin...",$48.00,462,95.0,strict_14_with_grace_period,4.07
2,6606,"Fab, private seattle urban cottage!","This tiny cottage is only 15x10, but it has ev...",Other neighborhoods,98103.0,Guesthouse,Entire home/apt,2,1.0,1,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",$90.00,139,92.0,strict_14_with_grace_period,1.14
3,9419,Glorious sun room w/ memory foambed,This beautiful double room features a magical ...,Other neighborhoods,98108.0,Apartment,Private room,2,3.0,1,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",$62.00,136,93.0,moderate,1.25
4,9460,Downtown Convention Center B&B -- Free Minibar,Take up a glass of wine and unwind on one of t...,Downtown,98101.0,Condominium,Private room,2,1.0,1,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$99.00,432,98.0,moderate,3.60


In [3]:
calendar_file = "../Project-ETL/Resources/calendar.csv"
calendar_df = pd.read_csv(calendar_file)
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2318,7/14/2019,f,$485.00,$485.00,4,1000
1,564428,7/14/2019,f,$59.00,$59.00,2,28
2,564428,7/15/2019,f,$59.00,$59.00,2,28
3,564428,7/16/2019,f,$59.00,$59.00,2,28
4,564428,7/17/2019,f,$59.00,$59.00,2,28


In [4]:
calendar_df['price'] = calendar_df['price'].str.replace(',', '')
calendar_df['price'] = calendar_df['price'].str.replace('$', '')
calendar_df['adjusted_price'] = calendar_df['adjusted_price'].str.replace(',', '')
calendar_df['adjusted_price'] = calendar_df['adjusted_price'].str.replace('$', '')
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2318,7/14/2019,f,485.00,485.00,4,1000
1,564428,7/14/2019,f,59.00,59.00,2,28
2,564428,7/15/2019,f,59.00,59.00,2,28
3,564428,7/16/2019,f,59.00,59.00,2,28
4,564428,7/17/2019,f,59.00,59.00,2,28


In [5]:
calendar_df['price'] = pd.to_numeric(calendar_df['price'])
calendar_df['adjusted_price'] = pd.to_numeric(calendar_df['adjusted_price'])

In [6]:
print (calendar_df.dtypes)

listing_id          int64
date               object
available          object
price             float64
adjusted_price    float64
minimum_nights      int64
maximum_nights      int64
dtype: object


In [7]:
calendar_df_variance = calendar_df['price'] - calendar_df['adjusted_price']

In [8]:
#calendar_df.append('calendar_df_variance')

In [9]:
calendar_df['price'].sum()

160411561.0

In [10]:
calendar_df['adjusted_price'].sum()

160335694.0

In [11]:
reviews_file = "../Project-ETL/Resources/reviews.csv"
reviews_df = pd.read_csv(reviews_file)
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2318,146,2008-09-15,2451,Kevin,1000 times better than staying at a hotel.
1,2318,126302712,2017-01-10,12332845,Jessica,"Our family (two couples, a two year old and an..."
2,2318,140977084,2017-04-01,4789466,Ivan,Top of the list locations we have stayed at! T...
3,2318,147262504,2017-04-25,55817131,Mike,"SUCH an awesome place. Very clean, quiet and s..."
4,2318,161806368,2017-06-18,113604590,Pete,We flew quite a distance to be at our only dau...


### Transform Listings DataFrame

In [12]:
# Create a filtered dataframe from specific columns
listings_cols = ["id", "neighbourhood_group_cleansed", "zipcode","property_type","room_type","amenities","number_of_reviews","review_scores_rating","cancellation_policy"]
listings_transformed= listings_df[listings_cols].copy()

# Rename the column headers
listings_transformed = listings_transformed.rename(columns={"id": "listing_id",
                                                          "neighbourhood_group_cleansed": "neighborhood",
                                                          })

listings_transformed.head()

,listing_id,neighborhood,zipcode,property_type,room_type,amenities,number_of_reviews,review_scores_rating,cancellation_policy
0,2318,Central Area,98122.0,House,Entire home/apt,"{Internet,Wifi,Kitchen,""Free parking on premis...",27,100.0,strict_14_with_grace_period
1,5682,Delridge,98106.0,Guest suite,Entire home/apt,"{TV,Internet,Wifi,""Free street parking"",Heatin...",462,95.0,strict_14_with_grace_period
2,6606,Other neighborhoods,98103.0,Guesthouse,Entire home/apt,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",139,92.0,strict_14_with_grace_period
3,9419,Other neighborhoods,98108.0,Apartment,Private room,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",136,93.0,moderate
4,9460,Downtown,98101.0,Condominium,Private room,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",432,98.0,moderate


### Transform Calendar DataFrame

In [13]:
# Create a filtered dataframe from specific columns
calendar_cols = ["listing_id", "adjusted_price", "minimum_nights","maximum_nights"]
calendar_transformed= calendar_df[calendar_cols].copy()

# Rename the column headers
calendar_transformed = calendar_transformed.rename(columns={"adjusted_price": "price",
                                                          })

calendar_transformed.head()

,listing_id,price,minimum_nights,maximum_nights
0,2318,485.0,4,1000
1,564428,59.0,2,28
2,564428,59.0,2,28
3,564428,59.0,2,28
4,564428,59.0,2,28


### Transform Reviews DataFrame

In [14]:
# Create a filtered dataframe from specific columns
reviews_cols = ["listing_id", "reviewer_id", "comments"]
reviews_transformed= reviews_df[reviews_cols].copy()

reviews_transformed.head()

,listing_id,reviewer_id,comments
0,2318,2451,1000 times better than staying at a hotel.
1,2318,12332845,"Our family (two couples, a two year old and an..."
2,2318,4789466,Top of the list locations we have stayed at! T...
3,2318,55817131,"SUCH an awesome place. Very clean, quiet and s..."
4,2318,113604590,We flew quite a distance to be at our only dau...


### Create database connection

In [15]:
connection_string = "postgres:postgres@localhost:5432/airbnb_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
#Confirm tables
engine.table_names()

['listings', 'calendar', 'reviews']

### Load DataFrames into database

In [17]:
listings_transformed.shape

(8921, 9)

In [18]:
calendar_transformed.shape

(1048575, 4)

In [19]:
reviews_transformed.shape

(427311, 3)

In [20]:
listings_transformed.to_sql(name='listings', con=engine, if_exists='append', index=False)

In [21]:
#reviews_transformed.to_csv('reviews_transformed.csv',index=False)

In [22]:
calendar_transformed.to_csv('calendar_transformed.csv',index=False)

In [25]:
reviews_transformed.to_sql(name='reviews', con=engine, if_exists='append', index=False)

In [ ]:
# @event.listens_for(engine, 'before_cursor_execute')
# def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
#     if executemany:
#         cursor.fast_executemany = True
#calendar_transformed.to_sql(name='calendar', con=engine, if_exists='append', index=False, chunksize = 10000)